In [301]:
import javalang
import csv
import sys
import itertools

from pipe import *

Attributes:
* method body
* return type
* argument list
* containing class name
* method name (label)

Attribute extensions:

In [8]:
code_snippet = """/*
 * Licensed to Elasticsearch under one or more contributor
 * license agreements. See the NOTICE file distributed with
 * this work for additional information regarding copyright
 * ownership. Elasticsearch licenses this file to you under
 * the Apache License, Version 2.0 (the "License"); you may
 * not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *    http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing,
 * software distributed under the License is distributed on an
 * "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
 * KIND, either express or implied.  See the License for the
 * specific language governing permissions and limitations
 * under the License.
 */

package org.elasticsearch.ingest.useragent;

import org.elasticsearch.common.cache.Cache;
import org.elasticsearch.common.cache.CacheBuilder;
import org.elasticsearch.ingest.useragent.UserAgentParser.Details;

import java.util.Objects;

class UserAgentCache {
    private final Cache<CompositeCacheKey, Details> cache;

    UserAgentCache(long cacheSize) {
        cache = CacheBuilder.<CompositeCacheKey, Details>builder().setMaximumWeight(cacheSize).build();
    }

    public Details get(String parserName, String userAgent) {
        return cache.get(new CompositeCacheKey(parserName, userAgent));
    }

    public void put(String parserName, String userAgent, Details details) {
        cache.put(new CompositeCacheKey(parserName, userAgent), details);
    }

    private static final class CompositeCacheKey {
        private final String parserName;
        private final String userAgent;

        CompositeCacheKey(String parserName, String userAgent) {
            this.parserName = parserName;
            this.userAgent = userAgent;
        }

        @Override
        public boolean equals(Object obj) {
            if(obj != null && obj instanceof CompositeCacheKey) {
                CompositeCacheKey s = (CompositeCacheKey)obj;
                return parserName.equals(s.parserName) && userAgent.equals(s.userAgent);
            }
            return false;
        }

        @Override
        public int hashCode() {
            return Objects.hash(parserName, userAgent);
        }
    }
}
"""

In [64]:
code_snippet_with_doc = '''/*
 * Licensed to Elasticsearch under one or more contributor
 * license agreements. See the NOTICE file distributed with
 * this work for additional information regarding copyright
 * ownership. Elasticsearch licenses this file to you under
 * the Apache License, Version 2.0 (the "License"); you may
 * not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *    http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing,
 * software distributed under the License is distributed on an
 * "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
 * KIND, either express or implied.  See the License for the
 * specific language governing permissions and limitations
 * under the License.
 */
package org.elasticsearch.gradle;

import java.io.File;

import javax.inject.Inject;

import org.gradle.api.DefaultTask;
import org.gradle.api.tasks.Input;
import org.gradle.api.tasks.Internal;
import org.gradle.api.tasks.TaskAction;
import org.gradle.internal.file.Chmod;

/**
 * Creates an empty directory.
 */
public class EmptyDirTask extends DefaultTask {

    private File dir;
    private int dirMode = 0755;

    /**
     * Creates an empty directory with the configured permissions.
     */
    @TaskAction
    public void create() {
        dir.mkdirs();
        getChmod().chmod(dir, dirMode);
    }

    @Inject
    public Chmod getChmod() {
        throw new UnsupportedOperationException();
    }

    @Internal
    public File getDir() {
        return dir;
    }

    @Input
    public String getDirPath() {
        return dir.getPath();
    }

    /**
     * @param dir The directory to create
     */
    public void setDir(File dir) {
        this.dir = dir;
    }

    /**
     * @param dir The path of the directory to create. Takes a String and coerces it to a file.
     */
    public void setDir(String dir) {
        this.dir = getProject().file(dir);
    }

    @Input
    public int getDirMode() {
        return dirMode;
    }

    /**
     * @param dirMode The permissions to apply to the new directory
     */
    public void setDirMode(int dirMode) {
        this.dirMode = dirMode;
    }

}
'''

In [89]:
big_file = '''/*
 * Licensed to Elasticsearch under one or more contributor
 * license agreements. See the NOTICE file distributed with
 * this work for additional information regarding copyright
 * ownership. Elasticsearch licenses this file to you under
 * the Apache License, Version 2.0 (the "License"); you may
 * not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *    http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing,
 * software distributed under the License is distributed on an
 * "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
 * KIND, either express or implied.  See the License for the
 * specific language governing permissions and limitations
 * under the License.
 */

package org.elasticsearch.client;

import org.elasticsearch.action.ActionListener;
import org.elasticsearch.client.ccr.CcrStatsRequest;
import org.elasticsearch.client.ccr.CcrStatsResponse;
import org.elasticsearch.client.ccr.DeleteAutoFollowPatternRequest;
import org.elasticsearch.client.ccr.FollowInfoRequest;
import org.elasticsearch.client.ccr.FollowInfoResponse;
import org.elasticsearch.client.ccr.FollowStatsRequest;
import org.elasticsearch.client.ccr.FollowStatsResponse;
import org.elasticsearch.client.ccr.ForgetFollowerRequest;
import org.elasticsearch.client.ccr.GetAutoFollowPatternRequest;
import org.elasticsearch.client.ccr.GetAutoFollowPatternResponse;
import org.elasticsearch.client.ccr.PauseAutoFollowPatternRequest;
import org.elasticsearch.client.ccr.PauseFollowRequest;
import org.elasticsearch.client.ccr.PutAutoFollowPatternRequest;
import org.elasticsearch.client.ccr.PutFollowRequest;
import org.elasticsearch.client.ccr.PutFollowResponse;
import org.elasticsearch.client.ccr.ResumeAutoFollowPatternRequest;
import org.elasticsearch.client.ccr.ResumeFollowRequest;
import org.elasticsearch.client.ccr.UnfollowRequest;
import org.elasticsearch.client.core.AcknowledgedResponse;
import org.elasticsearch.client.core.BroadcastResponse;

import java.io.IOException;
import java.util.Collections;

/**
 * A wrapper for the {@link RestHighLevelClient} that provides methods for
 * accessing the Elastic ccr related methods
 * <p>
 * See the <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-apis.html">
 * X-Pack Rollup APIs on elastic.co</a> for more information.
 */
public final class CcrClient {

    private final RestHighLevelClient restHighLevelClient;

    CcrClient(RestHighLevelClient restHighLevelClient) {
        this.restHighLevelClient = restHighLevelClient;
    }

    /**
     * Executes the put follow api, which creates a follower index and then the follower index starts following
     * the leader index.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-put-follow.html">
     * the docs</a> for more.
     *
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return the response
     * @throws IOException in case there is a problem sending the request or parsing back the response
     */
    public PutFollowResponse putFollow(PutFollowRequest request, RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
            request,
            CcrRequestConverters::putFollow,
            options,
            PutFollowResponse::fromXContent,
            Collections.emptySet()
        );
    }

    /**
     * Asynchronously executes the put follow api, which creates a follower index and then the follower index starts
     * following the leader index.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-put-follow.html">
     * the docs</a> for more.
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @param listener the listener to be notified upon request completion
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable putFollowAsync(PutFollowRequest request,
                                      RequestOptions options,
                                      ActionListener<PutFollowResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
            request,
            CcrRequestConverters::putFollow,
            options,
            PutFollowResponse::fromXContent,
            listener,
            Collections.emptySet()
        );
    }

    /**
     * Instructs a follower index to pause the following of a leader index.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-pause-follow.html">
     * the docs</a> for more.
     *
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return the response
     * @throws IOException in case there is a problem sending the request or parsing back the response
     */
    public AcknowledgedResponse pauseFollow(PauseFollowRequest request, RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
            request,
            CcrRequestConverters::pauseFollow,
            options,
            AcknowledgedResponse::fromXContent,
            Collections.emptySet()
        );
    }

    /**
     * Asynchronously instruct a follower index to pause the following of a leader index.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-pause-follow.html">
     * the docs</a> for more.
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @param listener the listener to be notified upon request completion
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable pauseFollowAsync(PauseFollowRequest request,
                                        RequestOptions options,
                                        ActionListener<AcknowledgedResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
            request,
            CcrRequestConverters::pauseFollow,
            options,
            AcknowledgedResponse::fromXContent,
            listener,
            Collections.emptySet());
    }

    /**
     * Instructs a follower index to resume the following of a leader index.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-resume-follow.html">
     * the docs</a> for more.
     *
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return the response
     * @throws IOException in case there is a problem sending the request or parsing back the response
     */
    public AcknowledgedResponse resumeFollow(ResumeFollowRequest request, RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
            request,
            CcrRequestConverters::resumeFollow,
            options,
            AcknowledgedResponse::fromXContent,
            Collections.emptySet()
        );
    }

    /**
     * Asynchronously instruct a follower index to resume the following of a leader index.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-resume-follow.html">
     * the docs</a> for more.
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @param listener the listener to be notified upon request completion
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable resumeFollowAsync(ResumeFollowRequest request,
                                         RequestOptions options,
                                         ActionListener<AcknowledgedResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
            request,
            CcrRequestConverters::resumeFollow,
            options,
            AcknowledgedResponse::fromXContent,
            listener,
            Collections.emptySet());
    }

    /**
     * Instructs a follower index to unfollow and become a regular index.
     * Note that index following needs to be paused and the follower index needs to be closed.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-unfollow.html">
     * the docs</a> for more.
     *
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return the response
     * @throws IOException in case there is a problem sending the request or parsing back the response
     */
    public AcknowledgedResponse unfollow(UnfollowRequest request, RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
            request,
            CcrRequestConverters::unfollow,
            options,
            AcknowledgedResponse::fromXContent,
            Collections.emptySet()
        );
    }

    /**
     * Asynchronously instructs a follower index to unfollow and become a regular index.
     * Note that index following needs to be paused and the follower index needs to be closed.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-unfollow.html">
     * the docs</a> for more.
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @param listener the listener to be notified upon request completion
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable unfollowAsync(UnfollowRequest request,
                                     RequestOptions options,
                                     ActionListener<AcknowledgedResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
            request,
            CcrRequestConverters::unfollow,
            options,
            AcknowledgedResponse::fromXContent,
            listener,
            Collections.emptySet()
        );
    }

    /**
     * Instructs an index acting as a leader index to forget the specified follower index.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-forget-follower.html">the docs</a> for more details
     * on the intended usage of this API.
     *
     * @param request the request
     * @param options the request options (e.g., headers), use {@link RequestOptions#DEFAULT} if the defaults are acceptable.
     * @return the response
     * @throws IOException if an I/O exception occurs while executing this request
     */
    public BroadcastResponse forgetFollower(final ForgetFollowerRequest request, final RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
                request,
                CcrRequestConverters::forgetFollower,
                options,
                BroadcastResponse::fromXContent,
                Collections.emptySet());
    }

    /**
     * Asynchronously instructs an index acting as a leader index to forget the specified follower index.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-forget-follower.html">the docs</a> for more details
     * on the intended usage of this API.
     * @param request the request
     * @param options the request options (e.g., headers), use {@link RequestOptions#DEFAULT} if the defaults are acceptable.
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable forgetFollowerAsync(
            final ForgetFollowerRequest request,
            final RequestOptions options,
            final ActionListener<BroadcastResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
                request,
                CcrRequestConverters::forgetFollower,
                options,
                BroadcastResponse::fromXContent,
                listener,
                Collections.emptySet());
    }

    /**
     * Stores an auto follow pattern.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-put-auto-follow-pattern.html">
     * the docs</a> for more.
     *
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return the response
     * @throws IOException in case there is a problem sending the request or parsing back the response
     */
    public AcknowledgedResponse putAutoFollowPattern(PutAutoFollowPatternRequest request, RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
            request,
            CcrRequestConverters::putAutoFollowPattern,
            options,
            AcknowledgedResponse::fromXContent,
            Collections.emptySet()
        );
    }

    /**
     * Asynchronously stores an auto follow pattern.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-put-auto-follow-pattern.html">
     * the docs</a> for more.
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @param listener the listener to be notified upon request completion
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable putAutoFollowPatternAsync(PutAutoFollowPatternRequest request,
                                                 RequestOptions options,
                                                 ActionListener<AcknowledgedResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
            request,
            CcrRequestConverters::putAutoFollowPattern,
            options,
            AcknowledgedResponse::fromXContent,
            listener,
            Collections.emptySet());
    }

    /**
     * Deletes an auto follow pattern.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-delete-auto-follow-pattern.html">
     * the docs</a> for more.
     *
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return the response
     * @throws IOException in case there is a problem sending the request or parsing back the response
     */
    public AcknowledgedResponse deleteAutoFollowPattern(DeleteAutoFollowPatternRequest request,
                                                        RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
            request,
            CcrRequestConverters::deleteAutoFollowPattern,
            options,
            AcknowledgedResponse::fromXContent,
            Collections.emptySet()
        );
    }

    /**
     * Asynchronously deletes an auto follow pattern.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-delete-auto-follow-pattern.html">
     * the docs</a> for more.
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @param listener the listener to be notified upon request completion
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable deleteAutoFollowPatternAsync(DeleteAutoFollowPatternRequest request,
                                                    RequestOptions options,
                                                    ActionListener<AcknowledgedResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
            request,
            CcrRequestConverters::deleteAutoFollowPattern,
            options,
            AcknowledgedResponse::fromXContent,
            listener,
            Collections.emptySet()
        );
    }

    /**
     * Gets an auto follow pattern.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-get-auto-follow-pattern.html">
     * the docs</a> for more.
     *
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return the response
     * @throws IOException in case there is a problem sending the request or parsing back the response
     */
    public GetAutoFollowPatternResponse getAutoFollowPattern(GetAutoFollowPatternRequest request,
                                                             RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
            request,
            CcrRequestConverters::getAutoFollowPattern,
            options,
            GetAutoFollowPatternResponse::fromXContent,
            Collections.emptySet()
        );
    }

    /**
     * Asynchronously gets an auto follow pattern.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-get-auto-follow-pattern.html">
     * the docs</a> for more.
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @param listener the listener to be notified upon request completion
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable getAutoFollowPatternAsync(GetAutoFollowPatternRequest request,
                                                 RequestOptions options,
                                                 ActionListener<GetAutoFollowPatternResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
            request,
            CcrRequestConverters::getAutoFollowPattern,
            options,
            GetAutoFollowPatternResponse::fromXContent,
            listener,
            Collections.emptySet()
        );
    }

    /**
     * Pauses an auto follow pattern.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-pause-auto-follow-pattern.html">
     * the docs</a> for more.
     *
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return the response
     * @throws IOException in case there is a problem sending the request or parsing back the response
     */
    public AcknowledgedResponse pauseAutoFollowPattern(PauseAutoFollowPatternRequest request, RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
            request,
            CcrRequestConverters::pauseAutoFollowPattern,
            options,
            AcknowledgedResponse::fromXContent,
            Collections.emptySet()
        );
    }

    /**
     * Asynchronously pauses an auto follow pattern.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-pause-auto-follow-pattern.html">
     * the docs</a> for more.
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @param listener the listener to be notified upon request completion
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable pauseAutoFollowPatternAsync(PauseAutoFollowPatternRequest request,
                                                   RequestOptions options,
                                                   ActionListener<AcknowledgedResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
            request,
            CcrRequestConverters::pauseAutoFollowPattern,
            options,
            AcknowledgedResponse::fromXContent,
            listener,
            Collections.emptySet());
    }

    /**
     * Resumes an auto follow pattern.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-resume-auto-follow-pattern.html">
     * the docs</a> for more.
     *
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return the response
     * @throws IOException in case there is a problem sending the request or parsing back the response
     */
    public AcknowledgedResponse resumeAutoFollowPattern(ResumeAutoFollowPatternRequest request, RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
            request,
            CcrRequestConverters::resumeAutoFollowPattern,
            options,
            AcknowledgedResponse::fromXContent,
            Collections.emptySet()
        );
    }

    /**
     * Asynchronously resumes an auto follow pattern.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-resume-auto-follow-pattern.html">
     * the docs</a> for more.
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @param listener the listener to be notified upon request completion
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable resumeAutoFollowPatternAsync(ResumeAutoFollowPatternRequest request,
                                                    RequestOptions options,
                                                    ActionListener<AcknowledgedResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
            request,
            CcrRequestConverters::resumeAutoFollowPattern,
            options,
            AcknowledgedResponse::fromXContent,
            listener,
            Collections.emptySet());
    }

    /**
     * Gets all CCR stats.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-get-stats.html">
     * the docs</a> for more.
     *
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return the response
     * @throws IOException in case there is a problem sending the request or parsing back the response
     */
    public CcrStatsResponse getCcrStats(CcrStatsRequest request,
                                        RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
            request,
            CcrRequestConverters::getCcrStats,
            options,
            CcrStatsResponse::fromXContent,
            Collections.emptySet()
        );
    }

    /**
     * Asynchronously gets all CCR stats.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-get-stats.html">
     * the docs</a> for more.
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable getCcrStatsAsync(CcrStatsRequest request,
                                        RequestOptions options,
                                        ActionListener<CcrStatsResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
            request,
            CcrRequestConverters::getCcrStats,
            options,
            CcrStatsResponse::fromXContent,
            listener,
            Collections.emptySet()
        );
    }

    /**
     * Gets follow stats for specific indices.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-get-follow-stats.html">
     * the docs</a> for more.
     *
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return the response
     * @throws IOException in case there is a problem sending the request or parsing back the response
     */
    public FollowStatsResponse getFollowStats(FollowStatsRequest request,
                                              RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
            request,
            CcrRequestConverters::getFollowStats,
            options,
            FollowStatsResponse::fromXContent,
            Collections.emptySet()
        );
    }

    /**
     * Asynchronously gets follow stats for specific indices.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-get-follow-stats.html">
     * the docs</a> for more.
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable getFollowStatsAsync(FollowStatsRequest request,
                                           RequestOptions options,
                                           ActionListener<FollowStatsResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
            request,
            CcrRequestConverters::getFollowStats,
            options,
            FollowStatsResponse::fromXContent,
            listener,
            Collections.emptySet()
        );
    }

    /**
     * Gets follow info for specific indices.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-get-follow-info.html">
     * the docs</a> for more.
     *
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return the response
     * @throws IOException in case there is a problem sending the request or parsing back the response
     */
    public FollowInfoResponse getFollowInfo(FollowInfoRequest request, RequestOptions options) throws IOException {
        return restHighLevelClient.performRequestAndParseEntity(
            request,
            CcrRequestConverters::getFollowInfo,
            options,
            FollowInfoResponse::fromXContent,
            Collections.emptySet()
        );
    }

    /**
     * Asynchronously gets follow info for specific indices.
     *
     * See <a href="https://www.elastic.co/guide/en/elasticsearch/reference/current/ccr-get-follow-info.html">
     * the docs</a> for more.
     * @param request the request
     * @param options the request options (e.g. headers), use {@link RequestOptions#DEFAULT} if nothing needs to be customized
     * @return cancellable that may be used to cancel the request
     */
    public Cancellable getFollowInfoAsync(FollowInfoRequest request,
                                          RequestOptions options,
                                          ActionListener<FollowInfoResponse> listener) {
        return restHighLevelClient.performRequestAsyncAndParseEntity(
            request,
            CcrRequestConverters::getFollowInfo,
            options,
            FollowInfoResponse::fromXContent,
            listener,
            Collections.emptySet()
        );
    }
}
'''

In [97]:
test_case = '''/*
 * Licensed to Elasticsearch under one or more contributor
 * license agreements. See the NOTICE file distributed with
 * this work for additional information regarding copyright
 * ownership. Elasticsearch licenses this file to you under
 * the Apache License, Version 2.0 (the "License"); you may
 * not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *    http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing,
 * software distributed under the License is distributed on an
 * "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
 * KIND, either express or implied.  See the License for the
 * specific language governing permissions and limitations
 * under the License.
 */

package org.elasticsearch.client;

import org.apache.http.Header;
import org.apache.http.HttpHost;
import org.apache.http.client.AuthCache;
import org.apache.http.impl.auth.BasicScheme;
import org.apache.http.impl.client.BasicAuthCache;
import org.apache.http.impl.nio.client.CloseableHttpAsyncClient;
import org.elasticsearch.client.RestClient.NodeTuple;

import java.io.IOException;
import java.net.URI;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.Collections;
import java.util.HashMap;
import java.util.Iterator;
import java.util.List;
import java.util.Map;
import java.util.concurrent.CountDownLatch;
import java.util.concurrent.TimeUnit;
import java.util.concurrent.atomic.AtomicInteger;
import java.util.concurrent.atomic.AtomicLong;
import java.util.function.Supplier;

import static java.util.Collections.singletonList;
import static org.hamcrest.Matchers.instanceOf;
import static org.junit.Assert.assertEquals;
import static org.junit.Assert.assertSame;
import static org.junit.Assert.assertThat;
import static org.junit.Assert.assertTrue;
import static org.junit.Assert.fail;
import static org.mockito.Mockito.mock;
import static org.mockito.Mockito.times;
import static org.mockito.Mockito.verify;

public class RestClientTests extends RestClientTestCase {

    public void testCloseIsIdempotent() throws IOException {
        List<Node> nodes = singletonList(new Node(new HttpHost("localhost", 9200)));
        CloseableHttpAsyncClient closeableHttpAsyncClient = mock(CloseableHttpAsyncClient.class);
        RestClient restClient = new RestClient(closeableHttpAsyncClient, new Header[0], nodes, null, null, null, false);
        restClient.close();
        verify(closeableHttpAsyncClient, times(1)).close();
        restClient.close();
        verify(closeableHttpAsyncClient, times(2)).close();
        restClient.close();
        verify(closeableHttpAsyncClient, times(3)).close();
    }

    public void testPerformAsyncWithUnsupportedMethod() throws Exception {
        final CountDownLatch latch = new CountDownLatch(1);
        try (RestClient restClient = createRestClient()) {
            restClient.performRequestAsync(new Request("unsupported", randomAsciiLettersOfLength(5)), new ResponseListener() {
                @Override
                public void onSuccess(Response response) {
                    throw new UnsupportedOperationException("onSuccess cannot be called when using a mocked http client");
                }

                @Override
                public void onFailure(Exception exception) {
                    try {
                        assertThat(exception, instanceOf(UnsupportedOperationException.class));
                        assertEquals("http method not supported: unsupported", exception.getMessage());
                    } finally {
                        latch.countDown();
                    }
                }
            });
            assertTrue("time out waiting for request to return", latch.await(1000, TimeUnit.MILLISECONDS));
        }
    }

    public void testPerformAsyncWithWrongEndpoint() throws Exception {
        final CountDownLatch latch = new CountDownLatch(1);
        try (RestClient restClient = createRestClient()) {
            restClient.performRequestAsync(new Request("GET", "::http:///"), new ResponseListener() {
                @Override
                public void onSuccess(Response response) {
                    throw new UnsupportedOperationException("onSuccess cannot be called when using a mocked http client");
                }

                @Override
                public void onFailure(Exception exception) {
                    try {
                        assertThat(exception, instanceOf(IllegalArgumentException.class));
                        assertEquals("Expected scheme name at index 0: ::http:///", exception.getMessage());
                    } finally {
                        latch.countDown();
                    }
                }
            });
            assertTrue("time out waiting for request to return", latch.await(1000, TimeUnit.MILLISECONDS));
        }
    }

    public void testBuildUriLeavesPathUntouched() {
        final Map<String, String> emptyMap = Collections.emptyMap();
        {
            URI uri = RestClient.buildUri("/foo$bar", "/index/type/id", emptyMap);
            assertEquals("/foo$bar/index/type/id", uri.getPath());
        }
        {
            URI uri = RestClient.buildUri("/", "/*", emptyMap);
            assertEquals("/*", uri.getPath());
        }
        {
            URI uri = RestClient.buildUri("/", "*", emptyMap);
            assertEquals("/*", uri.getPath());
        }
        {
            URI uri = RestClient.buildUri(null, "*", emptyMap);
            assertEquals("*", uri.getPath());
        }
        {
            URI uri = RestClient.buildUri("", "*", emptyMap);
            assertEquals("*", uri.getPath());
        }
        {
            URI uri = RestClient.buildUri(null, "/*", emptyMap);
            assertEquals("/*", uri.getPath());
        }
        {
            URI uri = RestClient.buildUri(null, "/foo$bar/ty/pe/i/d", emptyMap);
            assertEquals("/foo$bar/ty/pe/i/d", uri.getPath());
        }
        {
            URI uri = RestClient.buildUri(null, "/index/type/id", Collections.singletonMap("foo$bar", "x/y/z"));
            assertEquals("/index/type/id", uri.getPath());
            assertEquals("foo$bar=x/y/z", uri.getQuery());
        }
    }

    public void testSetNodesWrongArguments() throws IOException {
        try (RestClient restClient = createRestClient()) {
            restClient.setNodes(null);
            fail("setNodes should have failed");
        } catch (IllegalArgumentException e) {
            assertEquals("nodes must not be null or empty", e.getMessage());
        }
        try (RestClient restClient = createRestClient()) {
            restClient.setNodes(Collections.<Node>emptyList());
            fail("setNodes should have failed");
        } catch (IllegalArgumentException e) {
            assertEquals("nodes must not be null or empty", e.getMessage());
        }
        try (RestClient restClient = createRestClient()) {
            restClient.setNodes(Collections.singletonList((Node) null));
            fail("setNodes should have failed");
        } catch (NullPointerException e) {
            assertEquals("node cannot be null", e.getMessage());
        }
        try (RestClient restClient = createRestClient()) {
            restClient.setNodes(Arrays.asList(
                new Node(new HttpHost("localhost", 9200)),
                null,
                new Node(new HttpHost("localhost", 9201))));
            fail("setNodes should have failed");
        } catch (NullPointerException e) {
            assertEquals("node cannot be null", e.getMessage());
        }
    }

    public void testSetNodesPreservesOrdering() throws Exception {
        try (RestClient restClient = createRestClient()) {
            List<Node> nodes = randomNodes();
            restClient.setNodes(nodes);
            assertEquals(nodes, restClient.getNodes());
        }
    }

    private static List<Node> randomNodes() {
        int numNodes = randomIntBetween(1, 10);
        List<Node> nodes = new ArrayList<>(numNodes);
        for (int i = 0; i < numNodes; i++) {
            nodes.add(new Node(new HttpHost("host-" + i, 9200)));
        }
        return nodes;
    }

    public void testSetNodesDuplicatedHosts() throws Exception {
        try (RestClient restClient = createRestClient()) {
            int numNodes = randomIntBetween(1, 10);
            List<Node> nodes = new ArrayList<>(numNodes);
            Node node = new Node(new HttpHost("host", 9200));
            for (int i = 0; i < numNodes; i++) {
                nodes.add(node);
            }
            restClient.setNodes(nodes);
            assertEquals(1, restClient.getNodes().size());
            assertEquals(node, restClient.getNodes().get(0));
        }
    }

    public void testSelectHosts() throws IOException {
        Node n1 = new Node(new HttpHost("1"), null, null, "1", null, null);
        Node n2 = new Node(new HttpHost("2"), null, null, "2", null, null);
        Node n3 = new Node(new HttpHost("3"), null, null, "3", null, null);

        NodeSelector not1 = new NodeSelector() {
            @Override
            public void select(Iterable<Node> nodes) {
                for (Iterator<Node> itr = nodes.iterator(); itr.hasNext();) {
                    if ("1".equals(itr.next().getVersion())) {
                        itr.remove();
                    }
                }
            }

            @Override
            public String toString() {
                return "NOT 1";
            }
        };
        NodeSelector noNodes = new NodeSelector() {
            @Override
            public void select(Iterable<Node> nodes) {
                for (Iterator<Node> itr = nodes.iterator(); itr.hasNext();) {
                    itr.next();
                    itr.remove();
                }
            }

            @Override
            public String toString() {
                return "NONE";
            }
        };

        NodeTuple<List<Node>> nodeTuple = new NodeTuple<>(Arrays.asList(n1, n2, n3), null);

        Map<HttpHost, DeadHostState> emptyBlacklist = Collections.emptyMap();

        // Normal cases where the node selector doesn't reject all living nodes
        assertSelectLivingHosts(Arrays.asList(n1, n2, n3), nodeTuple, emptyBlacklist, NodeSelector.ANY);
        assertSelectLivingHosts(Arrays.asList(n2, n3), nodeTuple, emptyBlacklist, not1);

        /*
         * Try a NodeSelector that excludes all nodes. This should
         * throw an exception
         */
        {
            String message = "NodeSelector [NONE] rejected all nodes, living ["
                    + "[host=http://1, version=1], [host=http://2, version=2], "
                    + "[host=http://3, version=3]] and dead []";
            assertEquals(message, assertSelectAllRejected(nodeTuple, emptyBlacklist, noNodes));
        }

        // Mark all the nodes dead for a few test cases
        {
            final AtomicLong time = new AtomicLong(0L);
            Supplier<Long> timeSupplier = time::get;
            Map<HttpHost, DeadHostState> blacklist = new HashMap<>();
            blacklist.put(n1.getHost(), new DeadHostState(timeSupplier));
            blacklist.put(n2.getHost(), new DeadHostState(new DeadHostState(timeSupplier)));
            blacklist.put(n3.getHost(), new DeadHostState(new DeadHostState(new DeadHostState(timeSupplier))));

            /*
             * case when fewer nodeTuple than blacklist, won't result in any IllegalCapacityException
             */
            {
                NodeTuple<List<Node>> fewerNodeTuple = new NodeTuple<>(Arrays.asList(n1, n2), null);
                assertSelectLivingHosts(Arrays.asList(n1), fewerNodeTuple, blacklist, NodeSelector.ANY);
                assertSelectLivingHosts(Arrays.asList(n2), fewerNodeTuple, blacklist, not1);
            }

            /*
             * selectHosts will revive a single host regardless of
             * blacklist time. It'll revive the node that is closest
             * to being revived that the NodeSelector is ok with.
             */
            assertEquals(singletonList(n1), RestClient.selectNodes(nodeTuple, blacklist, new AtomicInteger(), NodeSelector.ANY));
            assertEquals(singletonList(n2), RestClient.selectNodes(nodeTuple, blacklist, new AtomicInteger(), not1));

            /*
             * Try a NodeSelector that excludes all nodes. This should
             * return a failure, but a different failure than when the
             * blacklist is empty so that the caller knows that all of
             * their nodes are blacklisted AND blocked.
             */
            String message = "NodeSelector [NONE] rejected all nodes, living [] and dead ["
                    + "[host=http://1, version=1], [host=http://2, version=2], "
                    + "[host=http://3, version=3]]";
            assertEquals(message, assertSelectAllRejected(nodeTuple, blacklist, noNodes));

            /*
             * Now lets wind the clock forward, past the timeout for one of
             * the dead nodes. We should return it.
             */
            time.set(new DeadHostState(timeSupplier).getDeadUntilNanos());
            assertSelectLivingHosts(Arrays.asList(n1), nodeTuple, blacklist, NodeSelector.ANY);

            /*
             * But if the NodeSelector rejects that node then we'll pick the
             * first on that the NodeSelector doesn't reject.
             */
            assertSelectLivingHosts(Arrays.asList(n2), nodeTuple, blacklist, not1);

            /*
             * If we wind the clock way into the future, past any of the
             * blacklist timeouts then we function as though the nodes aren't
             * in the blacklist at all.
             */
            time.addAndGet(DeadHostState.MAX_CONNECTION_TIMEOUT_NANOS);
            assertSelectLivingHosts(Arrays.asList(n1, n2, n3), nodeTuple, blacklist, NodeSelector.ANY);
            assertSelectLivingHosts(Arrays.asList(n2, n3), nodeTuple, blacklist, not1);
        }
    }

    private void assertSelectLivingHosts(List<Node> expectedNodes, NodeTuple<List<Node>> nodeTuple,
            Map<HttpHost, DeadHostState> blacklist, NodeSelector nodeSelector) throws IOException {
        int iterations = 1000;
        AtomicInteger lastNodeIndex = new AtomicInteger(0);
        assertEquals(expectedNodes, RestClient.selectNodes(nodeTuple, blacklist, lastNodeIndex, nodeSelector));
        // Calling it again rotates the set of results
        for (int i = 1; i < iterations; i++) {
            Collections.rotate(expectedNodes, 1);
            assertEquals("iteration " + i, expectedNodes,
                    RestClient.selectNodes(nodeTuple, blacklist, lastNodeIndex, nodeSelector));
        }
    }

    /**
     * Assert that {@link RestClient#selectNodes} fails on the provided arguments.
     * @return the message in the exception thrown by the failure
     */
    private static String assertSelectAllRejected( NodeTuple<List<Node>> nodeTuple,
            Map<HttpHost, DeadHostState> blacklist, NodeSelector nodeSelector) {
        try {
            RestClient.selectNodes(nodeTuple, blacklist, new AtomicInteger(0), nodeSelector);
            throw new AssertionError("expected selectHosts to fail");
        } catch (IOException e) {
            return e.getMessage();
        }
    }

    private static RestClient createRestClient() {
        List<Node> nodes = Collections.singletonList(new Node(new HttpHost("localhost", 9200)));
        return new RestClient(mock(CloseableHttpAsyncClient.class), new Header[] {}, nodes, null, null, null, false);
    }

    public void testRoundRobin() throws IOException {
        int numNodes = randomIntBetween(2, 10);
        AuthCache authCache = new BasicAuthCache();
        List<Node> nodes = new ArrayList<>(numNodes);
        for (int i = 0; i < numNodes; i++) {
            Node node = new Node(new HttpHost("localhost", 9200 + i));
            nodes.add(node);
            authCache.put(node.getHost(), new BasicScheme());
        }
        NodeTuple<List<Node>> nodeTuple = new NodeTuple<>(nodes, authCache);

        //test the transition from negative to positive values
        AtomicInteger lastNodeIndex = new AtomicInteger(-numNodes);
        assertNodes(nodeTuple, lastNodeIndex, 50);
        assertEquals(-numNodes + 50, lastNodeIndex.get());

        //test the highest positive values up to MAX_VALUE
        lastNodeIndex.set(Integer.MAX_VALUE - numNodes * 10);
        assertNodes(nodeTuple, lastNodeIndex, numNodes * 10);
        assertEquals(Integer.MAX_VALUE, lastNodeIndex.get());

        //test the transition from MAX_VALUE to MIN_VALUE
        //this is the only time where there is most likely going to be a jump from a node
        //to another one that's not necessarily the next one.
        assertEquals(Integer.MIN_VALUE, lastNodeIndex.incrementAndGet());
        assertNodes(nodeTuple, lastNodeIndex, 50);
        assertEquals(Integer.MIN_VALUE + 50, lastNodeIndex.get());
    }

    private static void assertNodes(NodeTuple<List<Node>> nodeTuple, AtomicInteger lastNodeIndex, int runs) throws IOException {
        int distance = lastNodeIndex.get() % nodeTuple.nodes.size();
        /*
         * Collections.rotate is not super intuitive: distance 1 means that the last element will become the first and so on,
         * while distance -1 means that the second element will become the first and so on.
         */
        int expectedOffset = distance > 0 ? nodeTuple.nodes.size() - distance : Math.abs(distance);
        for (int i = 0; i < runs; i++) {
            Iterable<Node> selectedNodes = RestClient.selectNodes(nodeTuple, Collections.<HttpHost, DeadHostState>emptyMap(),
                    lastNodeIndex, NodeSelector.ANY);
            List<Node> expectedNodes = nodeTuple.nodes;
            int index = 0;
            for (Node actualNode : selectedNodes) {
                Node expectedNode = expectedNodes.get((index + expectedOffset) % expectedNodes.size());
                assertSame(expectedNode, actualNode);
                index++;
            }
            expectedOffset--;
            if (expectedOffset < 0) {
                expectedOffset += nodeTuple.nodes.size();
            }
        }
    }
}
'''

In [130]:
code_snippet.splitlines()[2:]

[' * license agreements. See the NOTICE file distributed with',
 ' * this work for additional information regarding copyright',
 ' * ownership. Elasticsearch licenses this file to you under',
 ' * the Apache License, Version 2.0 (the "License"); you may',
 ' * not use this file except in compliance with the License.',
 ' * You may obtain a copy of the License at',
 ' *',
 ' *    http://www.apache.org/licenses/LICENSE-2.0',
 ' *',
 ' * Unless required by applicable law or agreed to in writing,',
 ' * software distributed under the License is distributed on an',
 ' * "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY',
 ' * KIND, either express or implied.  See the License for the',
 ' * specific language governing permissions and limitations',
 ' * under the License.',
 ' */',
 '',
 'package org.elasticsearch.ingest.useragent;',
 '',
 'import org.elasticsearch.common.cache.Cache;',
 'import org.elasticsearch.common.cache.CacheBuilder;',
 'import org.elasticsearch.ingest.useragent.Us

In [264]:
# cut everything between (including) the body's opening and closing brackets
def get_text_between_positions(text_lines, start_line, start_column, end_line, end_column):
    inner_lines = text_lines[start_line:(end_line - 1)]
    splitted_first_line = text_lines[start_line - 1].split('{')
    first_line_text = '{' + (splitted_first_line[1] if len(splitted_first_line) > 1 else '')
    last_line_text = text_lines[end_line - 1]

    return ' '.join([s.strip() for s in [first_line_text] + inner_lines + [last_line_text]]).strip()

In [232]:
a, b ='public boolean equals(Object obj) { '.split('{')
b

' '

In [204]:
['a', 'b'] + ['c', 'd']

['a', 'b', 'c', 'd']

In [246]:
test_case.splitlines()[329]

'            Map<HttpHost, DeadHostState> blacklist, NodeSelector nodeSelector) throws IOException {'

In [265]:
print(get_text_between_positions(test_case.splitlines(), 329, 13, 340, 5))

{ Map<HttpHost, DeadHostState> blacklist, NodeSelector nodeSelector) throws IOException { int iterations = 1000; AtomicInteger lastNodeIndex = new AtomicInteger(0); assertEquals(expectedNodes, RestClient.selectNodes(nodeTuple, blacklist, lastNodeIndex, nodeSelector)); // Calling it again rotates the set of results for (int i = 1; i < iterations; i++) { Collections.rotate(expectedNodes, 1); assertEquals("iteration " + i, expectedNodes, RestClient.selectNodes(nodeTuple, blacklist, lastNodeIndex, nodeSelector)); } }


In [236]:
def get_matching_closing_bracket_position(tokens):
    brackets = tokens | where(lambda token: token.value == '{' or token.value == '}')
    started_counting = False
    counter = 0
    end_position = None
    
    for bracket in brackets:
        if bracket.value == '{':
            started_counting = True
            counter += 1
        elif bracket.value == '}':
            counter -= 1

        if started_counting and counter == 0:
            end_position = bracket.position
            break
    
    return end_position

def get_method_body(text_lines, tokens, line, column):
    filtered_tokens = tokens \
        | skip_while(lambda token: token.position.line < line) \
        | skip_while(lambda token: token.position.column < column) \
        | skip_while(lambda token: token.value != '{')
    
    # TODO: can we omit the generator copying
    filtered_tokens, filtered_tokens_copy = itertools.tee(filtered_tokens)
    closing_bracket_position = get_matching_closing_bracket_position(filtered_tokens_copy)
    
    return get_text_between_positions(
        text_lines,
        start_line=line,
        start_column=column,
        end_line=closing_bracket_position.line,
        end_column=closing_bracket_position.column
    )
    
#     body_tokens = filtered_tokens \
#         | take_while(lambda token: token.position.line <= closing_bracket_position.line or \
#                                    (token.position.line == closing_bracket_position.line and token.position.column == closing_bracket_position.column)) \
#         | map(lambda token: token.value)

#     return ' '.join(body_tokens)

In [288]:
a = 2
f'asdf: {a}'

'asdf: 2'

In [ ]:
class Method(collections.namedtuple('Method', [
    'name',
    'class_name',
    'return_type',
    'arguments',
    'body',
    'documentation',
    'modifiers',
    'annotations'
])):
    def __repr__(self):
        return f'{self.name}| of {self.suit}'

In [298]:
def get_type_name(node):
    # isinstance(node.return_type, javalang.tree.BasicType)
    return 'void' if node is None else node.name

def get_arguments(node):
    return list(node.parameters | map(lambda param: [param.name, get_type_name(param.type)]))

def get_attributes(class_name, text_lines, tokens, method_node):    
    if not isinstance(method_node, javalang.tree.MethodDeclaration):
        raise ValueError('The given node is not of type MethodDeclaration')
    
    node = method_node
    
    return {
        'name': node.name,
        'class_name': class_name,
        'body': get_method_body(text_lines, tokens, line=node.position.line, column=node.position.column),
        'return_type': get_type_name(node.return_type),
        'arguments': get_arguments(node),
        'documentation': repr(node.documentation),
        'modifiers': list(node.modifiers),
        'annotations': list(node.annotations | map(lambda annotation: annotation.name)),
    }

In [284]:
def parse_source_code(source_code_text):
    tree = javalang.parse.parse(source_code_text)
    tokens = list(javalang.tokenizer.tokenize(source_code_text))
    text_lines = source_code_text.splitlines()

    parsed_methods = []
    
    for t, class_node in tree.filter(javalang.tree.ClassDeclaration):
        class_name = class_node.name
        parsed_methods_in_class = list(class_node.methods \
            | map(lambda tree_node_tuple: get_attributes(class_name, text_lines, tokens, tree_node_tuple)))
        parsed_methods = parsed_methods + parsed_methods_in_class
        
    return parsed_methods

In [285]:
file_path = '../data/repos/elasticsearch-master/plugins/repository-s3/src/main/java/org/elasticsearch/repositories/s3/SocketAccess.java'
with open(file_path) as file:
    print(parse_source_code(file.read()))

[{'name': 'doPrivileged', 'class_name': 'SocketAccess', 'body': '{ SpecialPermission.check(); return AccessController.doPrivileged(operation); }', 'return_type': 'T', 'arguments': [['operation', 'PrivilegedAction']], 'documentation': None, 'modifiers': {'static', 'public'}, 'annotations': []}, {'name': 'doPrivilegedIOException', 'class_name': 'SocketAccess', 'body': '{ SpecialPermission.check(); try { return AccessController.doPrivileged(operation); } catch (PrivilegedActionException e) { throw (IOException) e.getCause(); } }', 'return_type': 'T', 'arguments': [['operation', 'PrivilegedExceptionAction']], 'documentation': None, 'modifiers': {'static', 'public'}, 'annotations': []}, {'name': 'doPrivilegedVoid', 'class_name': 'SocketAccess', 'body': '{ SpecialPermission.check(); AccessController.doPrivileged((PrivilegedAction<Void>) () -> { action.run(); return null; }); }', 'return_type': 'void', 'arguments': [['action', 'Runnable']], 'documentation': None, 'modifiers': {'static', 'publ

In [303]:
fieldnames = [
    'name',
    'class_name',
    'return_type',
    'arguments',
    'body',
    'documentation',
    'modifiers',
    'annotations'
]

parsed_data = parse_source_code(test_case)

# parsed_data

# with open('output.csv', 'w') as csvfile:
writer = csv.DictWriter(sys.stdout, fieldnames=fieldnames)
writer.writeheader()
for row in parsed_data:
    writer.writerow(row)

name,class_name,return_type,arguments,body,documentation,modifiers,annotations
testCloseIsIdempotent,RestClientTests,void,[],"{ List<Node> nodes = singletonList(new Node(new HttpHost(""localhost"", 9200))); CloseableHttpAsyncClient closeableHttpAsyncClient = mock(CloseableHttpAsyncClient.class); RestClient restClient = new RestClient(closeableHttpAsyncClient, new Header[0], nodes, null, null, null, false); restClient.close(); verify(closeableHttpAsyncClient, times(1)).close(); restClient.close(); verify(closeableHttpAsyncClient, times(2)).close(); restClient.close(); verify(closeableHttpAsyncClient, times(3)).close(); }",None,['public'],[]
testPerformAsyncWithUnsupportedMethod,RestClientTests,void,[],"{ final CountDownLatch latch = new CountDownLatch(1); try (RestClient restClient = createRestClient()) { restClient.performRequestAsync(new Request(""unsupported"", randomAsciiLettersOfLength(5)), new ResponseListener() { @Override public void onSuccess(Response response) { throw new Unsu

In [120]:
'\n'.join(test_case.splitlines())

'/*\n * Licensed to Elasticsearch under one or more contributor\n * license agreements. See the NOTICE file distributed with\n * this work for additional information regarding copyright\n * ownership. Elasticsearch licenses this file to you under\n * the Apache License, Version 2.0 (the "License"); you may\n * not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *    http://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing,\n * software distributed under the License is distributed on an\n * "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY\n * KIND, either express or implied.  See the License for the\n * specific language governing permissions and limitations\n * under the License.\n */\n\npackage org.elasticsearch.client;\n\nimport org.apache.http.Header;\nimport org.apache.http.HttpHost;\nimport org.apache.http.client.AuthCache;\nimport org.apache.http.impl.auth.BasicScheme;

In [270]:
for t, node in tree.filter(javalang.tree.ClassDeclaration):
    print(node.name, len(node.methods))

UserAgentCache 2
CompositeCacheKey 2


In [9]:
tokens = javalang.tokenizer.tokenize('System.out.println("Hello " + "world");')
list(tokens)

[Identifier "System" line 1, position 1,
 Separator "." line 1, position 7,
 Identifier "out" line 1, position 8,
 Separator "." line 1, position 11,
 Identifier "println" line 1, position 12,
 Separator "(" line 1, position 19,
 String ""Hello "" line 1, position 20,
 Operator "+" line 1, position 29,
 String ""world"" line 1, position 31,
 Separator ")" line 1, position 38,
 Separator ";" line 1, position 39]